<a href="https://colab.research.google.com/github/Shivansh-datascience/RAG_Agent_system/blob/main/RAG_AGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing all packages required

In [8]:
import langchain
import pypdf
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import logging
import warnings
warnings.filterwarnings("ignore")
import pickle

In [2]:
pip install pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.7 MB/s eta 0:00:00


In [3]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 11.7 MB/s eta 0:00:00


In [4]:
pip install langchain

In [5]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20


In [6]:
pip install rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.3 MB/s eta 0:00:00


In [22]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 39.6 MB/s eta 0:00:00


# Loading the pdf file

In [10]:
#file path contents form google colab environment
file_path = "/content/meidtations (1).pdf"
extract_images = True
password = None


def load_pdf_file(file_path,extract_images,password):
  logging.basicConfig(level=logging.INFO)   #tracking each event for RAG system
  loader = PyPDFLoader(file_path=file_path,extract_images=extract_images,password=password)   #created an loader object for laoding the contents
  try:
    pdf_contents = loader.load()   #load the contents through (loader object)
    return pdf_contents
  except Exception as error:
    return f" error conifgured: {error}"


#calling the above function
pdf_contents = load_pdf_file(file_path,extract_images,password)
print(f" length of pdf file {len(pdf_contents)}")

 length of pdf file 128


In [11]:
print(pdf_contents)

[Document(metadata={'producer': 'macOS Version 14.2.1 (Build 23C71) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20240110035838Z00'00'", 'title': 'https:/www.maximusveritas.com/wp-content/uploads/2017/09/Marcus-Aurelius-Meditations', 'author': 'Arshad Kazi', 'moddate': "D:20240110035838Z00'00'", 'source': '/content/meidtations (1).pdf', 'total_pages': 128, 'page': 0, 'page_label': '1'}, page_content="Marcus Aurelius' Meditations - tr. Casaubon v. 8.16, uploaded to www.philaletheians.co.uk, 14 July 2013 \nPage 1 of 128 \nThe meditations of \nMarcus Aurelius Antoninus \nOriginally translated by Meric Casaubon \n \nAbout this edition \nMarcus Aurelius Antoninus Augustus was Emperor of Rome from 161 to his death, \nthe last of the “Five Good Emperors.” He was nephew, son-in-law, and adoptive son \nof Antonius Pius. Marcus Aurelius was one of the most important Stoic philosophers, \ncited by H.P. Blavatsky amongst famous classic sages and writers such as Plato, Eu-\nripides, 

length of contents for each pages

In [12]:
for page in pdf_contents:
  print(f"page {page.page_content}")

page Marcus Aurelius' Meditations - tr. Casaubon v. 8.16, uploaded to www.philaletheians.co.uk, 14 July 2013 
Page 1 of 128 
The meditations of 
Marcus Aurelius Antoninus 
Originally translated by Meric Casaubon 
 
About this edition 
Marcus Aurelius Antoninus Augustus was Emperor of Rome from 161 to his death, 
the last of the “Five Good Emperors.” He was nephew, son-in-law, and adoptive son 
of Antonius Pius. Marcus Aurelius was one of the most important Stoic philosophers, 
cited by H.P. Blavatsky amongst famous classic sages and writers such as Plato, Eu-
ripides, Socrates, Aristophanes, Pindar, Plutarch, Isocrates, Diodorus, Cicero, and 
Epictetus.1 
This edition was originally translated out of the Greek by Meric Casaubon in 1634 as 
“The Golden Book of Marcus Aurelius,” with an Introduction by W.H.D. Rouse. It was 
subsequently edited by Ernest Rhys. London: J.M. Dent & Co; New York: E.P. Dutton 
& Co, 1906; Everyman’s Library. 
                                            1 Cf. 

# Splitting pdf contents into chunks for faster processing

In [13]:
#defining the chunk size
chunk_size = 200
chunk_overlap = 50    #splitting the text after each 100 steps

def split_documents(pdf_contents,chunk_size,chunk_overlap):
  logging.basicConfig(level=logging.INFO)
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap,separators=["\n\n","\n"])
  try:
    documents = text_splitter.split_documents(pdf_contents)
  except Exception as error:
    return f"error configured: {error}"
  finally:
    return documents

documents = split_documents(pdf_contents,chunk_size,chunk_overlap)
print(f"length of documents {len(documents)}")

length of documents 2636


In [14]:
for document in documents:
  print(f"document {document.page_content}")

document Marcus Aurelius' Meditations - tr. Casaubon v. 8.16, uploaded to www.philaletheians.co.uk, 14 July 2013 
Page 1 of 128 
The meditations of 
Marcus Aurelius Antoninus
document The meditations of 
Marcus Aurelius Antoninus 
Originally translated by Meric Casaubon 
 
About this edition 
Marcus Aurelius Antoninus Augustus was Emperor of Rome from 161 to his death,
document the last of the “Five Good Emperors.” He was nephew, son-in-law, and adoptive son 
of Antonius Pius. Marcus Aurelius was one of the most important Stoic philosophers,
document cited by H.P. Blavatsky amongst famous classic sages and writers such as Plato, Eu-
ripides, Socrates, Aristophanes, Pindar, Plutarch, Isocrates, Diodorus, Cicero, and 
Epictetus.1
document Epictetus.1 
This edition was originally translated out of the Greek by Meric Casaubon in 1634 as 
“The Golden Book of Marcus Aurelius,” with an Introduction by W.H.D. Rouse. It was
document subsequently edited by Ernest Rhys. London: J.M. Dent & Co; Ne

In [15]:
#lowering all the text
for document in documents:
  document.page_content = document.page_content.lower()
  print(f"document {document.page_content}")

document marcus aurelius' meditations - tr. casaubon v. 8.16, uploaded to www.philaletheians.co.uk, 14 july 2013 
page 1 of 128 
the meditations of 
marcus aurelius antoninus
document the meditations of 
marcus aurelius antoninus 
originally translated by meric casaubon 
 
about this edition 
marcus aurelius antoninus augustus was emperor of rome from 161 to his death,
document the last of the “five good emperors.” he was nephew, son-in-law, and adoptive son 
of antonius pius. marcus aurelius was one of the most important stoic philosophers,
document cited by h.p. blavatsky amongst famous classic sages and writers such as plato, eu-
ripides, socrates, aristophanes, pindar, plutarch, isocrates, diodorus, cicero, and 
epictetus.1
document epictetus.1 
this edition was originally translated out of the greek by meric casaubon in 1634 as 
“the golden book of marcus aurelius,” with an introduction by w.h.d. rouse. it was
document subsequently edited by ernest rhys. london: j.m. dent & co; ne

# converting paragraph text into sentences

In [16]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [17]:
def converting_into_sentences(document):
  logging.basicConfig(level=logging.INFO)
  new_pages = []  #for storing all new contents into these
  for pages in document:
    sent_tokenize(pages.page_content,language="english")
    new_pages.append(pages)
  return new_pages

new_pages = converting_into_sentences(documents)
print(new_pages)

[Document(metadata={'producer': 'macOS Version 14.2.1 (Build 23C71) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20240110035838Z00'00'", 'title': 'https:/www.maximusveritas.com/wp-content/uploads/2017/09/Marcus-Aurelius-Meditations', 'author': 'Arshad Kazi', 'moddate': "D:20240110035838Z00'00'", 'source': '/content/meidtations (1).pdf', 'total_pages': 128, 'page': 0, 'page_label': '1'}, page_content="marcus aurelius' meditations - tr. casaubon v. 8.16, uploaded to www.philaletheians.co.uk, 14 july 2013 \npage 1 of 128 \nthe meditations of \nmarcus aurelius antoninus"), Document(metadata={'producer': 'macOS Version 14.2.1 (Build 23C71) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20240110035838Z00'00'", 'title': 'https:/www.maximusveritas.com/wp-content/uploads/2017/09/Marcus-Aurelius-Meditations', 'author': 'Arshad Kazi', 'moddate': "D:20240110035838Z00'00'", 'source': '/content/meidtations (1).pdf', 'total_pages': 128, 'page': 0, 'page_label': '1'}, page_

sent tokens helps to improve the model query performance more faster

# creating an embedding vector for storing the text into vector database

In [18]:
#using hugging face embedding
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"   #using sentence transformers model for embedding all the test into vector format

def create_embedding_vector(embedding_model):
  logging.basicConfig(level=logging.INFO)
  try:
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
  except Exception as error:
    return f"error configured: {error}"
  finally:
    return embeddings

embedder = create_embedding_vector(embedding_model)
print(embedder)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


# creating an vector database for storing the page contents

In [23]:
#creating an vector database
def create_vector_store(new_pages,embedder):
  logging.basicConfig(level=logging.INFO)
  try:
    faiss_database = FAISS.from_documents(new_pages,embedding = embedder)   #storing all pages into vector database with sentence transformers embedding
    return faiss_database
  except Exception as error:
    return f"error configured: {error}"

faiss_database = create_vector_store(new_pages,embedder)
print(faiss_database)

# testing the vector database with actual query input

In [24]:
query = input("enter your query")
response = faiss_database.similarity_search(query)
print(f" response : {response[0]}")

enter your querycomplete the senetnce The ideal Wise Man is sufficient unto himself in all things (αυταρκής);
 response : page_content='page 7 of 128 
tue may act. the ideal wise man is sufficient unto himself in all things (αυταρκής); 
and knowing these truths, he will be happy even when stretched upon the rack. it is' metadata={'producer': 'macOS Version 14.2.1 (Build 23C71) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20240110035838Z00'00'", 'title': 'https:/www.maximusveritas.com/wp-content/uploads/2017/09/Marcus-Aurelius-Meditations', 'author': 'Arshad Kazi', 'moddate': "D:20240110035838Z00'00'", 'source': '/content/meidtations (1).pdf', 'total_pages': 128, 'page': 6, 'page_label': '7'}


model is repsonding best base on similiarity search through vector database

# Saving the vector database

In [28]:
def save_vector_database(faiss_database,embedder,new_pages):
  logging.info("Saving the vector database")
  try:
    retriver = faiss_database.as_retriever()
    return retriver
  except Exception as error:
    return f"error configured: {error}"

retriver = save_vector_database(faiss_database,embedder,new_pages)
print(retriver)

tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f5a6afea390> search_kwargs={}


In [32]:
#saving the vector database with embedding model
vector_file_location = "/content/faiss_database.pkl"

try:
  faiss_database.save_local(vector_file_location)
  print("vector database saved successfully")
except Exception as error:
  print(f"error configured: {error}")

vector database saved successfully


# Testing the query reponse through retriver

In [26]:
query = input("enter your query")
response = retriver.get_relevant_documents(query)
print(f" response : {response[0]}")

enter your querycomplete the senetnce The ideal Wise Man is sufficient unto himself in all things (αυταρκής)
 response : page_content='page 7 of 128 
tue may act. the ideal wise man is sufficient unto himself in all things (αυταρκής); 
and knowing these truths, he will be happy even when stretched upon the rack. it is' metadata={'producer': 'macOS Version 14.2.1 (Build 23C71) Quartz PDFContext', 'creator': 'Safari', 'creationdate': "D:20240110035838Z00'00'", 'title': 'https:/www.maximusveritas.com/wp-content/uploads/2017/09/Marcus-Aurelius-Meditations', 'author': 'Arshad Kazi', 'moddate': "D:20240110035838Z00'00'", 'source': '/content/meidtations (1).pdf', 'total_pages': 128, 'page': 6, 'page_label': '7'}


retriver is also getting the response regarding the pdf rag agent

In [36]:
#opening the vector database from colab environment
colab_file_location = "/content/faiss_database.pkl"
try:
    # Attempt to load with deserialization allowed
    faiss_db = FAISS.load_local(colab_file_location, embedder, allow_dangerous_deserialization=True)
    print("Vector database loaded successfully.")
except ValueError as e:
  raise e

Vector database loaded successfully.


In [37]:
#running the query through faiss database
query = input("Enter your respone")
response_1 = faiss_db.similarity_search(query=query)
print(f" repsone generated : {response_1[0]}")

Enter your respone#opening the vector database from colab environment colab_file_location = "/content/faiss_database.pkl" try:     # Attempt to load with deserialization allowed     faiss_db = FAISS.load_local(colab_file_location, embedder, allow_dangerous_deserialization=True)     print("Vector database loaded successfully.") except ValueError as e:     if "allow_dangerous_deserialization" in str(e):         # Handle the specific ValueError related to deserialization         print("WARNING: Loading from an untrusted source could be risky. Proceed with caution!")         user_input = input("Are you sure you trust the source of this file? (yes/no): ")         if user_input.lower() == "yes":             faiss_db = FAISS.load_local(colab_file_location, embedder, allow_dangerous_deserialization=True)             print("Vector database loaded successfully (with user confirmation).")         else:             print("Vector database loading aborted due to security concerns.")             # Yo